In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os

import random
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set()

### Analyze product combiantions probabilty

In [2]:
frames = []
temp1 = pd.read_csv('data/2019-Dec.csv')
temp2 = pd.read_csv('data/2019-Nov.csv')
temp3 = pd.read_csv('data/2019-Oct.csv')
temp4 = pd.read_csv('data/2020-Feb.csv')
temp5 = pd.read_csv('data/2020-Jan.csv')

frames.append(temp1)
frames.append(temp2)
frames.append(temp3)
frames.append(temp4)
frames.append(temp5)

df=pd.concat(frames,ignore_index=True,sort=False)

print("The dataframe has {} rows and {} columns.\n".format(df.shape[0],df.shape[1]))
print("Shown below are the first 3 rows of the dataframe:\n")
pd.set_option('display.max_columns', 100)
df.head()

The dataframe has 20692840 rows and 9 columns.

Shown below are the first 3 rows of the dataframe:



,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [3]:
df_purchased=df.loc[df.event_type=='purchase',:]

df_purchased=df_purchased.drop(columns=['category_code','price','brand','category_id','event_type'])


df_purchased['event_time']=pd.to_datetime(df_purchased['event_time'],infer_datetime_format=True)

nullcolumns=df_purchased.isnull().sum()
nullnumbers=len(nullcolumns[nullcolumns!=0])
print("After data selection and cleansing, the dataframe has {} rows, {} columns, and {} null value.\n".format(df_purchased.shape[0],df_purchased.shape[1],nullnumbers))
df_purchased.head()

After data selection and cleansing, the dataframe has 1287007 rows, 4 columns, and 0 null value.



,event_time,product_id,user_id,user_session
557,2019-12-01 00:17:50+00:00,4554,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
558,2019-12-01 00:17:50+00:00,4600,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
559,2019-12-01 00:17:50+00:00,4607,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
560,2019-12-01 00:17:50+00:00,4640,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
561,2019-12-01 00:17:50+00:00,4689,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9


In [4]:
df_purchased = df_purchased.dropna()
df_purchased.head()

,event_time,product_id,user_id,user_session
557,2019-12-01 00:17:50+00:00,4554,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
558,2019-12-01 00:17:50+00:00,4600,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
559,2019-12-01 00:17:50+00:00,4607,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
560,2019-12-01 00:17:50+00:00,4640,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9
561,2019-12-01 00:17:50+00:00,4689,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9


In [5]:
new_df1 = df_purchased.groupby(['user_id','user_session'])['product_id'].apply(list).reset_index()
new_df1

,user_id,user_session,product_id
0,9794320,30d70cc3-86ee-4b5d-879c-40f06132163c,"[5812033, 5724621, 5809910, 5811668]"
1,10079204,f873d16b-4efa-4e92-81be-92948aef2ff5,"[5766379, 5614842]"
2,10280338,5a8ef64a-9215-4c88-ad5f-b3bab4e03fb3,"[5863562, 5878917, 5903722, 5903741, 5903767, ..."
3,10280338,9728269f-2014-4d85-a95a-0dab99c4851e,"[5877585, 5878917, 5900337, 5900634, 5901659, ..."
4,10280338,bcc9bfa0-8a71-485d-b315-297b0038b4c2,"[5809910, 5809911, 5809912, 5862544, 5870648, ..."
...,...,...,...
155612,622065819,a6c280be-ad01-413d-818e-a818f3ac3558,"[5659905, 5659911, 5697535, 5699414]"
155613,622066161,389680bd-69c6-4364-b549-651f82f8ffce,"[5909239, 5912323, 5912428, 5914129]"
155614,622067983,93a3b8e4-f726-4cdc-8d18-12d4c6f23b59,"[5622080, 5622678, 5667105, 5689728, 5796004, ..."
155615,622069477,8d85bde1-9586-4a39-a8ed-ecc59ea083a0,[5924419]


In [9]:
top_sequences = new_df1['product_id'].value_counts().to_frame(name='count')
top_sequences

,count
[5790689],190
[5877454],157
[5677043],137
[5692527],131
[5793261],130
...,...
"[5584844, 5887254, 5847573, 5773159, 5763379, 5759490]",1
"[27762, 27764, 5624537, 5624540, 5627419]",1
"[39956, 5820723, 5820752, 5881774]",1
"[5854897, 39956, 5858956, 5875369, 5875393, 5875405, 5875406, 5875413, 5875416, 5875424, 5875797, 5879015, 5881774, 5839412, 5817680, 5793704, 43188, 5883315]",1


In [10]:
top_sequences.to_csv('product_combination_counts.csv', header=False)

In [17]:
combinations = pd.read_csv('product_combination_counts.csv', delimiter=',')
combinations.columns = ['product_combination', 'count']
combinations

,product_combination,count
0,[5877454],157
1,[5677043],137
2,[5692527],131
3,[5793261],130
4,[5764716],125
...,...,...
141779,"[5584844, 5887254, 5847573, 5773159, 5763379, ...",1
141780,"[27762, 27764, 5624537, 5624540, 5627419]",1
141781,"[39956, 5820723, 5820752, 5881774]",1
141782,"[5854897, 39956, 5858956, 5875369, 5875393, 58...",1


In [25]:
combinations['probability'] = combinations['count']/combinations['count'].sum()
combinations

,product_combination,count,probability
0,[5877454],157,0.001010
1,[5677043],137,0.000881
2,[5692527],131,0.000843
3,[5793261],130,0.000836
4,[5764716],125,0.000804
...,...,...,...
141779,"[5584844, 5887254, 5847573, 5773159, 5763379, ...",1,0.000006
141780,"[27762, 27764, 5624537, 5624540, 5627419]",1,0.000006
141781,"[39956, 5820723, 5820752, 5881774]",1,0.000006
141782,"[5854897, 39956, 5858956, 5875369, 5875393, 58...",1,0.000006


In [41]:
# combinations.to_csv('product_combinations_probabilities.csv', index=False)

In [37]:
# combinations['product_combination'] = combinations['product_combination'].strip('[]').split(',')
combinations['product_combination'] = [x.strip('[]').split(',') for x in combinations['product_combination']]
combinations

,product_combination,count,probability
0,[5877454],157,0.001010
1,[5677043],137,0.000881
2,[5692527],131,0.000843
3,[5793261],130,0.000836
4,[5764716],125,0.000804
...,...,...,...
141779,"[5584844, 5887254, 5847573, 5773159, 57633...",1,0.000006
141780,"[27762, 27764, 5624537, 5624540, 5627419]",1,0.000006
141781,"[39956, 5820723, 5820752, 5881774]",1,0.000006
141782,"[5854897, 39956, 5858956, 5875369, 5875393...",1,0.000006


In [47]:
combinations.head()

,product_combination,count,probability
0,[5877454],157,0.001010
1,[5677043],137,0.000881
2,[5692527],131,0.000843
3,[5793261],130,0.000836
4,[5764716],125,0.000804


In [44]:
only_one_product = combinations.loc[combinations.product_combination.map(len) == 1]
more_than_one_product = combinations.loc[combinations.product_combination.map(len) > 1]

In [46]:
more_than_one_product

,product_combination,count,probability
23,"[5809910, 5809912]",58,0.000373
45,"[5814516, 5814517]",40,0.000257
47,"[5677043, 5697463]",38,0.000244
58,"[5751383, 5751422]",35,0.000225
60,"[5809910, 5809911]",34,0.000219
...,...,...,...
141778,"[5873430, 5864611, 5759486, 5651975, 5304]",1,0.000006
141779,"[5584844, 5887254, 5847573, 5773159, 57633...",1,0.000006
141780,"[27762, 27764, 5624537, 5624540, 5627419]",1,0.000006
141781,"[39956, 5820723, 5820752, 5881774]",1,0.000006


In [53]:
more_than_one_product = more_than_one_product.loc[more_than_one_product['count'] > 1]
more_than_one_product

,product_combination,count,probability
23,"[5809910, 5809912]",58,0.000373
45,"[5814516, 5814517]",40,0.000257
47,"[5677043, 5697463]",38,0.000244
58,"[5751383, 5751422]",35,0.000225
60,"[5809910, 5809911]",34,0.000219
...,...,...,...
3238,"[5814534, 4621]",2,0.000013
3239,"[5809910, 5816171]",2,0.000013
3240,"[5304, 5802439, 5810136, 5836517, 5836522,...",2,0.000013
3243,"[5749735, 5751383]",2,0.000013


In [54]:
more_than_one_product['probability'] = more_than_one_product['count']/more_than_one_product['count'].sum()
more_than_one_product

/tmp/ipykernel_150160/3766310827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_than_one_product['probability'] = more_than_one_product['count']/more_than_one_product['count'].sum()


,product_combination,count,probability
23,"[5809910, 5809912]",58,0.015761
45,"[5814516, 5814517]",40,0.010870
47,"[5677043, 5697463]",38,0.010326
58,"[5751383, 5751422]",35,0.009511
60,"[5809910, 5809911]",34,0.009239
...,...,...,...
3238,"[5814534, 4621]",2,0.000543
3239,"[5809910, 5816171]",2,0.000543
3240,"[5304, 5802439, 5810136, 5836517, 5836522,...",2,0.000543
3243,"[5749735, 5751383]",2,0.000543


In [61]:
more_than_one_product = more_than_one_product.reset_index()
more_than_one_product = more_than_one_product.drop(columns=['index'])
more_than_one_product

,level_0,product_combination,count,probability
0,0,"[5809910, 5809912]",58,0.015761
1,1,"[5814516, 5814517]",40,0.010870
2,2,"[5677043, 5697463]",38,0.010326
3,3,"[5751383, 5751422]",35,0.009511
4,4,"[5809910, 5809911]",34,0.009239
...,...,...,...,...
1276,1276,"[5814534, 4621]",2,0.000543
1277,1277,"[5809910, 5816171]",2,0.000543
1278,1278,"[5304, 5802439, 5810136, 5836517, 5836522,...",2,0.000543
1279,1279,"[5749735, 5751383]",2,0.000543


In [106]:
# more_than_one_product.to_csv('final_product_combinations.csv', index=False)

In [105]:
more_than_one_product[more_than_one_product['product_combination'].apply(lambda x: '5823970' in x)]

,product_combination,count,probability
196,"[5823969, 5823970]",3,0.000815
210,"[5815731, 5823970]",3,0.000815
587,"[5820270, 5823970, 5855120]",2,0.000543
935,"[5708154, 5751422, 5808322, 5808330, 5823970]",2,0.000543
956,"[5751383, 5751422, 5820270, 5823970]",2,0.000543
987,"[5820270, 5823970]",2,0.000543
1145,"[5815730, 5823970]",2,0.000543
1248,"[5587738, 5686925, 5700037, 5774349, 5809297, ...",2,0.000543
1264,"[5751422, 5820270, 5823970]",2,0.000543


In [109]:
def find_recomendations(product_id):
    recomendation = []
    results = more_than_one_product[more_than_one_product['product_combination'].apply(lambda x: product_id in x)]
    for i in results['product_combination']:
        for j in i:
            if j not in recomendation and j != product_id:
                recomendation.append(j)
                if len(recomendation) == 10:
                    break
        else:
            continue
        break
    return recomendation 

In [110]:
find_recomendations('5823970')

['5823969',
 '5815731',
 '5820270',
 '5855120',
 '5708154',
 '5751422',
 '5808322',
 '5808330',
 '5751383',
 '5815730']